In [1]:
from utils import *
from rich import print
import os
from numba import jit, njit
import copy
import numpy as np
import pandas as pd

In [2]:
# load h5py data
data = load_data('../data/new_cell.ims')

In [3]:
data.keys()


<KeysViewHDF5 ['DataSet', 'DataSet1', 'DataSet10', 'DataSet11', 'DataSet12', 'DataSet13', 'DataSet14', 'DataSet2', 'DataSet3', 'DataSet4', 'DataSet5', 'DataSet6', 'DataSet7', 'DataSet8', 'DataSet9', 'DataSetEvents', 'DataSetEvents1', 'DataSetEvents10', 'DataSetEvents11', 'DataSetEvents12', 'DataSetEvents13', 'DataSetEvents14', 'DataSetEvents2', 'DataSetEvents3', 'DataSetEvents4', 'DataSetEvents5', 'DataSetEvents6', 'DataSetEvents7', 'DataSetEvents8', 'DataSetEvents9', 'DataSetInfo', 'DataSetInfo1', 'DataSetInfo10', 'DataSetInfo11', 'DataSetInfo12', 'DataSetInfo13', 'DataSetInfo14', 'DataSetInfo2', 'DataSetInfo3', 'DataSetInfo4', 'DataSetInfo5', 'DataSetInfo6', 'DataSetInfo7', 'DataSetInfo8', 'DataSetInfo9', 'DataSetTimes', 'DataSetTimes1', 'DataSetTimes10', 'DataSetTimes11', 'DataSetTimes12', 'DataSetTimes13', 'DataSetTimes14', 'DataSetTimes2', 'DataSetTimes3', 'DataSetTimes4', 'DataSetTimes5', 'DataSetTimes6', 'DataSetTimes7', 'DataSetTimes8', 'DataSetTimes9', 'Scene', 'Scene8', 'Thum

In [4]:
data.get('Scene').get('Content').keys()

<KeysViewHDF5 ['Cells0', 'ImageMasks0', 'MegaSurfaces0', 'MegaSurfaces1']>

In [10]:
data.get('Scene8').get('Content').get('Cells0').keys()

<KeysViewHDF5 ['Category', 'Cells', 'Factor', 'FactorList', 'LabelColor', 'LabelColorData', 'LabelColorLabelGroupNames', 'LabelColorLabelValues', 'LabelGroupNames', 'LabelSetLabelIDs', 'LabelSetObjectIDs', 'LabelSets', 'LabelValues', 'MaskImage', 'Nuclei', 'StatisticsType', 'StatisticsValue', 'StatisticsValueTimeOffset', 'Time', 'TimeBegin']>

In [28]:
data.get('Scene8').get('Content').get('Cells0').get('Nuclei').keys()


<KeysViewHDF5 ['MainTrackSegmentTable', 'MainTrackSegmentTable_Focus', 'Nucleus', 'NucleusCellOffset', 'NucleusTimeOffset', 'TrackSegment0', 'TrackSegment0_Focus']>

In [45]:
np.array(data.get('Scene8').get('Content').get('Cells0').get('Nuclei')['Nucleus'])[1]

(100000320, 32719, 17664, 0)

In [41]:
np.array(data.get('Scene8').get('Content').get('Cells0').get('Nuclei')['NucleusCellOffset'])

array([(    1,     0,     1), (    2,     1,     2),
       (    3,     2,     3), ..., (16574, 15879, 15880),
       (16575, 15880, 15881), (16576, 15881, 15882)],
      dtype=[('ID', '<i8'), ('IndexBegin', '<i8'), ('IndexEnd', '<i8')])

In [29]:
g = [None] * 15882

r = np.array(data.get('Scene8').get('Content').get('Cells0').get('Nuclei').get('Nucleus'))

for i in range(15882):
    
    g[i] = r[i][2]

In [32]:
min(g)

17305

In [36]:
## can we get surfaces by modifying following function
def get_object_names(full_data_file: h5py.File, search_for: str) -> list:
    """
    _summary_

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        search_for (str): string containing full or partial filename to search for

    Returns:
        (list): a list of all the object names that match search_for parameter
    """
    

    values = full_data_file.get('Scene').get('Content').keys()
    storage = list()
    for item in values:
        if len(re.findall(search_for, item)):
            storage.append(item)
    return storage

# test function -- working
out = get_object_names(data, 'Cells')
print(out)

['Cells0']

In [37]:
# once we know the object names we want to extract data from use following function
# this function doesnt grab any data values, just names
def get_statistics_names(full_data_file: h5py.File, object_name: str) -> dict:
    """
    for a given object_name, extracts the statistics names and ids into a dict
    ex: statistics name = mean intensity, associated id=404

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        dict: a dict where the keys=unique stats ID, value=static name
    """

    # get object specific data
    obj_specific_data = full_data_file['Scene8']['Content'][object_name]
    
    # rearrange data
    statistics_name = np.asarray(obj_specific_data['StatisticsType'])
    statistics_name = pd.DataFrame(statistics_name)
    
    # extract statistics names
    stats_name = statistics_name['Name']
    
    # extract statistics ID names
    stats_type = statistics_name['ID']
    
    # combine stats type and stats names
    return  dict(zip(stats_type, stats_name))

# test -- working
out_stats_names = get_statistics_names(data, 'Cells0')
out_stats_names


{413: b'Cell Area',
 672: b'Cell BoundingBoxAA Length X',
 673: b'Cell BoundingBoxAA Length Y',
 674: b'Cell BoundingBoxAA Length Z',
 675: b'Cell BoundingBoxOO Length A',
 676: b'Cell BoundingBoxOO Length B',
 677: b'Cell BoundingBoxOO Length C',
 678: b'Cell Center of Homogeneous Mass X',
 679: b'Cell Center of Homogeneous Mass Y',
 680: b'Cell Center of Homogeneous Mass Z',
 681: b'Cell Center of Image Mass X',
 682: b'Cell Center of Image Mass X',
 683: b'Cell Center of Image Mass X',
 684: b'Cell Center of Image Mass X',
 990: b'Cell Center of Image Mass X',
 685: b'Cell Center of Image Mass Y',
 686: b'Cell Center of Image Mass Y',
 687: b'Cell Center of Image Mass Y',
 688: b'Cell Center of Image Mass Y',
 991: b'Cell Center of Image Mass Y',
 689: b'Cell Center of Image Mass Z',
 690: b'Cell Center of Image Mass Z',
 691: b'Cell Center of Image Mass Z',
 692: b'Cell Center of Image Mass Z',
 992: b'Cell Center of Image Mass Z',
 414: b'Cell Cytoplasm Area',
 693: b'Cell Cytopla

In [27]:
#np.savetxt('test.txt', list(out_stats_names.values()), fmt='%s')

In [28]:
# once we have the statistics names we can get numerical statistics values
# for each object id within the specified object
def get_stats_values(full_data_file: h5py.File, object_name: str) -> pd.DataFrame:
    """
    for a given object_name, extracts the statistics values for all object ids
    within the object

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        pd.DataFrame: a pandas data frame that contains information about each object id
        where each object id has a stats id and associated stats value.
    """
    obj_specific_stats = full_data_file.get('Scene8').get('Content')[object_name]['StatisticsValue']
    obj_specific_stats = np.asarray(obj_specific_stats)
    return pd.DataFrame(obj_specific_stats)

out = get_stats_values(data, 'Cells0').drop('ID_Time', axis=1)
print(out)

ID_Object  ID_StatisticsType         Value
0               -1                179      0.000000
1               -1                180      0.000000
2               -1                181  16577.000000
3               -1                182  15882.000000
4                0                413   2115.452637
...            ...                ...           ...
3173500  100016607                439    336.423096
3173501  100027732                439    367.477509
3173502  100016399                439    631.440247
3173503         -1                411  16577.000000
3173504         -1                412  15882.000000

[3173505 rows x 3 columns]

In [29]:
len(out['ID_Object'])

3173505

In [14]:
extracted_stats = out.groupby('ID_Object')[
    ['ID_StatisticsType', 'Value']].apply(lambda x: x.set_index('ID_StatisticsType').to_dict(orient='dict')).to_dict()
extracted_stats = {k: v['Value'] for k, v in extracted_stats.items()} # clean up step for line above

In [16]:
extracted_stats[1]

{3501: 610.707763671875,
 3502: 10.234375,
 3503: 4.55078125,
 3504: 32.00152587890625,
 3505: 3.814706802368164,
 3506: 9.053071975708008,
 3507: 28.016679763793945,
 3508: 65381.99609375,
 3509: 42356.77734375,
 3510: 17.762041091918945,
 3511: 65381.2265625,
 3512: 65382.421875,
 3513: 65382.359375,
 3514: 65381.44140625,
 3515: 42356.94140625,
 3516: 42356.5859375,
 3517: 42356.37890625,
 3518: 42356.765625,
 3519: 13.57917594909668,
 3520: 13.84041976928711,
 3521: 17.814804077148438,
 3522: 15.258231163024902,
 3523: 1121.3994140625,
 3290: 57.0,
 3291: 37.0,
 3292: 12.0,
 3293: 31.0,
 3294: 37.07692337036133,
 3295: 14.307692527770996,
 3296: 4.615384578704834,
 3297: 17.538461685180664,
 3298: 21.0,
 3299: 3.0,
 3300: 0.0,
 3301: 6.0,
 3524: 10.484138488769531,
 3525: 11.598306655883789,
 3526: 4.179408550262451,
 3527: 7.880017280578613,
 3528: 482.0,
 3529: 186.0,
 3530: 60.0,
 3531: 228.0,
 3532: 13.0,
 3302: 67.2835693359375,
 3533: 77903.1640625,
 3534: 0.0,
 3535: 0.0,
 3

In [ ]:
def invert_stats_dict(stats_dict: dict=None):
    '''
    creates a inverted_stats_dict --> [Statistics Type: Numeric Value]
    '''
    
    # sort the dict
    sorted_dict_key = sorted(stats_dict.keys())
    stats_dict = {key: str(stats_dict[key]).strip('b')[1:-1] for key in sorted_dict_key }
    
    # create an empty dictionary
    storage = {}
    
    for key in stats_dict.keys():
        # if the word is not in the new storage dict
        if stats_dict[key] not in storage.keys():
            storage[stats_dict[key]] = key
        else:
            # get the value inside the key 
            current_value = storage[stats_dict[key]]
            
            # if its a single value create a dict else create a dict
            if type(current_value) != dict:
                # then its the first value
                storage[stats_dict[key]] = {}
                storage[stats_dict[key]]['channel_1'] = current_value
                
                # current value
                storage[stats_dict[key]]['channel_2'] = key
            else:
                # get the length of the dict
                count = len(current_value.keys())
                # updated count
                count += 1 
                # create new key
                new_key = f"channel_{count}"
                current_value[new_key] = key
                storage[stats_dict[key]] = current_value
                
    return storage
# we can directly use this to invert the dict
# this inverted dict can be used to create the final excel file

In [ ]:
inverted_dict =  invert_stats_dict(out_stats_names)
categories = [
    'Position X',
    'Position Y',
    'Position Z',
    'Intensity Mean_channel_1',
    'Intensity Mean_channel_2',
    'Intensity Mean_channel_3',
    'Intensity Mean_channel_4',
    'Intensity Mean_channel_5',
    'Volume'
]

In [ ]:
# flatten dict 
from collections.abc import MutableMapping

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
inverted_flatten = flatten(inverted_dict)
# can we provide an editable list before anything starts so he can manually select which ones he wants
# that way the names match exactly because I control it. 

# reverse dict again, key=num, value=name
final_dict = {v: k for k,v in inverted_flatten.items()}

# # can we use the categories reqested to create a list of numbers we dont want?
# del_list = [None] * (len(inverted_flatten) - len(categories))
# counter = 0
# for idx, (k, v) in enumerate(inverted_flatten.items()):
    
#     if k not in categories and v != None:
#         del_list[counter] = v
#         counter += 1
    

In [ ]:
# map what the user wants to numeric values
user_request_numeric = [inverted_flatten[x] for x in categories]

# we can look at the final dict where key=num, value=name to filter
final_dict_copy = copy.deepcopy(final_dict)
for value in user_request_numeric:
    final_dict_copy.pop(value)
    
new_del_list = list(final_dict_copy.keys())

In [ ]:
print(len(final_dict_copy.keys()), len(final_dict.keys()), len(user_request_numeric))

In [ ]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=del_list, axis=1)
print(f)


In [ ]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=[None], axis=1)
print(f)s

In [ ]:
# clean up df by changing number to names and rearranging columns
columns = f.columns
print(columns)

new_names = {key: final_dict[key] for key in columns}
print(new_names)

# rename dict


In [ ]:
final_df = f.rename(new_names, axis=1)
print(final_df.head())

In [ ]:
# create the final id column
final_df['ID'] = final_df.index


# rearrange
final_order = [
    'ID',
    'Position X',
    'Position Y',
    'Position Z',
    'Intensity Mean_channel_1',
    'Intensity Mean_channel_2',
    'Intensity Mean_channel_3',
    'Intensity Mean_channel_4',
    'Intensity Mean_channel_5',
]

final = final_df[final_order]
print(final)